# DAT405 Introduction to Data Science and AI 
## 2021-2022, Reading Period 1
## Assignment 4: Spam classification using Naïve Bayes 
There will be an overall grade for this assignment. To get a pass grade (grade 5), you need to pass items 1-3 below. To receive higher grades, finish items 4 and 5 as well. 

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 7zip (https://www.7-zip.org/download.html) to decompress the data.



In [7]:
#Download and extract data
!mkdir datasets
%cd datasets
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2 
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2 
!tar -xjf 20021010_easy_ham.tar.bz2 
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2
%cd ..

mkdir: cannot create directory ‘datasets’: File exists
/home/islandengineer/Dev/Chalmers/dat405/assignment_4/datasets
--2021-09-22 15:54:21--  https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1677144 (1.6M) [application/x-bzip2]
Saving to: ‘20021010_easy_ham.tar.bz2’

20021010_easy_ham.t 100%[===================>]   1.60M  3.63MB/s    in 0.4s    

2021-09-22 15:54:21 (3.63 MB/s) - ‘20021010_easy_ham.tar.bz2’ saved [1677144/1677144]

--2021-09-22 15:54:21--  https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HT

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [8]:
!ls -lah

total 372K
drwxr-xr-x 3 islandengineer islandengineer 4.0K Sep 22 12:14 .
drwxr-xr-x 7 islandengineer islandengineer 4.0K Sep 22 11:34 ..
-rw-r--r-- 1 islandengineer islandengineer  19K Sep 22 15:53 DAT405_Assignment4.ipynb
-rw-r--r-- 1 islandengineer islandengineer 337K Sep 22 11:33 DAT405_Assignment4.pdf
drwxr-xr-x 5 islandengineer islandengineer 4.0K Sep 22 15:54 datasets


### 1. Preprocessing: 
1.	Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text. Further down (in the higher-grade part), you will be asked to filter out the headers and footers. 

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [10]:
def extract_emails(directories, class_name):
  rows = []
  for directory in directories:
    for file in os.listdir(directory):
      with open(os.path.join(directory,file), encoding='latin-1') as f:
        if f is not None:
          rows.append({'email': file, 'content': f.read(), 'class': class_name})
  return pd.DataFrame(rows)

# extract emails and add class information
df_ham = extract_emails(['./datasets/easy_ham', 'datasets/hard_ham'], 'ham')
df_spam = extract_emails(['./datasets/spam'], 'spam')

# join the two dataframes
df_emails = pd.concat([df_ham,df_spam])

df_emails.sample(5)

,email,content,class
1104,1617.75b61ae83ecff7b45a80cac6cc8c9192,From spamassassin-devel-admin@lists.sourceforg...,ham
986,1233.f19ee9e6ffb986dd0dd2691401e237ae,From exmh-workers-admin@redhat.com Wed Oct 2...,ham
1729,1023.909ebf9dc802a8f97b0d722ad443dd42,From fork-admin@xent.com Thu Oct 10 12:34:51 ...,ham
1333,2537.2769f2836293c4c9105e6a89fc14fd8e,From rssfeeds@jmason.org Thu Oct 10 12:32:18 ...,ham
56,0255.98492176173fd6951086bb7af9e0008a,From ilug-admin@linux.ie Wed Aug 28 10:48:25 ...,ham


2.	We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`)

In [11]:
# pre-processing code here
X_train, X_test, y_train, y_test = train_test_split(df_emails['content'], df_emails['class'], train_size=0.8, random_state=0, shuffle=True)

print('all emails:', df_emails.shape)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

all emails: (3302, 3)
X_train: (2641,)
X_test: (661,)
y_train: (2641,)
y_test: (661,)


### 2. Write a Python program that: 
1.	Uses four datasets (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (e.g. Sklearn) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem
- Multinomial Naive Bayes  
- Bernoulli Naive Bayes. 

Please inspect the documentation to ensure input to the classifiers is appropriate. Discuss the differences between these two classifiers. 





#### Multinomial Naive Bayes

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# instantiate a Count Vectorizer and fit for training data
cv = CountVectorizer()
X_train_vector = cv.fit_transform(X_train)
X_test_vector = cv.transform(X_test)

# fit Multinomial Naive Bayes model using training data
multinomial_naive_bayes = MultinomialNB().fit(X_train_vector, y_train)

# make predictions on the test data vector
y_pred = multinomial_naive_bayes.predict(X_test_vector)

y_pred[:10]

array(['ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham',
       'ham'], dtype='<U4')

#### Bernoulli Naive Bayes

`[insert discussion]`

### 3.Run your program on 
-	Spam versus easy-ham 
-	Spam versus hard-ham.

In [ ]:
#Code to report results here

###4.	To avoid classification based on common and uninformative words it is common to filter these out. 

**a.** Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. 

**b.** Use the parameters in Sklearn’s `CountVectorizer` to filter out these words. Update the program from point 3 and run it on your data and report your results.

You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you. Argue for your decision-making.


###5. Eeking out further performance
Filter out the headers and footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
-	Does the result improve from 3 and 4? 
- The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies? 
- What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages? 

Re-estimate your classifier using `fit_prior` parameter set to `false`, and answer the following questions:
- What does this parameter mean?
- How does this alter the predictions? Discuss why or why not.

### What to report and how to hand in.

- You will need to clearly report all results in the notebook in a clear and appropriate way, either using plots or code output (f.x. "print statements"). 
- The notebook must be reproducible, that means, we must be able to use the `Run all` function from the `Runtime` menu and reproduce all your results. **Please check this before handing in.** 
- Save the notebook and share a link to the notebook (Press share in upper left corner, and use `Get link` option. **Please make sure to allow all with the link to open and edit.**
- Edits made after submission deadline will be ignored, graders will recover the last saved version before deadline from the revisions history.
- **Please make sure all cells are executed and all the output is clearly readable/visible to anybody opening the notebook.**